# 101. 総合演習：空間の帰納バイアスを統合する

## 学習目標

このノートブックでは、Unit 0.4全体を振り返り、以下を統合的に理解します：

1. **畳み込み**の本質的理解
2. **受容野**とネットワーク設計
3. **帰納バイアス**の意味と限界
4. **セグメンテーション**アーキテクチャの設計思想

## 目次

1. [カリキュラムの振り返り](#section1)
2. [核心概念の統合](#section2)
3. [設計原則のまとめ](#section3)
4. [チェックリスト](#section4)
5. [次へのステップ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

<a id="section1"></a>
## 1. カリキュラムの振り返り

In [ ]:
def visualize_curriculum_map():
    """カリキュラム全体のマップを可視化"""
    fig, ax = plt.subplots(figsize=(18, 12))
    
    sections = [
        ('A. カーネル物理学\n(80-85)', 0.1, 0.8, 'lightblue', 
         ['畳み込みの直感', '数学的定義', 'NumPy実装', '古典フィルタ', 'カーネル可視化']),
        ('B. 受容野\n(86-89)', 0.35, 0.8, 'lightgreen',
         ['受容野入門', '深さと受容野', 'ダウンサンプリング', '3DGSとの類似性']),
        ('C. 帰納バイアス\n(90-95)', 0.6, 0.8, 'lightyellow',
         ['帰納バイアス入門', '重み共有', '並進等変性', 'CNN vs MLP', 'CNNの限界', 'ViT/MLP-Mixer']),
        ('D. セグメンテーション\n(96-100)', 0.35, 0.35, 'lightcoral',
         ['セマンティックセグメンテーション', 'U-Net', 'FPN', 'スキップ接続', '応用事例']),
        ('E. 統合\n(101-102)', 0.6, 0.35, 'lavender',
         ['総合演習', '未来への展望']),
    ]
    
    for name, x, y, color, topics in sections:
        rect = plt.Rectangle((x, y - 0.15), 0.2, 0.25, 
                            facecolor=color, edgecolor='gray', linewidth=2)
        ax.add_patch(rect)
        ax.text(x + 0.1, y + 0.08, name, ha='center', va='center', 
               fontsize=11, fontweight='bold')
        
        topic_text = '\n'.join([f'• {t}' for t in topics[:4]])
        if len(topics) > 4:
            topic_text += f'\n  + {len(topics)-4}項目'
        ax.text(x + 0.1, y - 0.08, topic_text, ha='center', va='center', 
               fontsize=8)
    
    # 矢印（学習の流れ）
    arrows = [
        ((0.3, 0.8), (0.35, 0.8)),   # A → B
        ((0.55, 0.8), (0.6, 0.8)),   # B → C
        ((0.45, 0.65), (0.45, 0.5)), # B → D
        ((0.7, 0.65), (0.7, 0.5)),   # C → E
        ((0.55, 0.35), (0.6, 0.35)), # D → E
    ]
    
    for start, end in arrows:
        ax.annotate('', xy=end, xytext=start,
                   arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
    
    ax.set_xlim(0, 0.95)
    ax.set_ylim(0.1, 1)
    ax.axis('off')
    ax.set_title('Unit 0.4 カリキュラムマップ：空間の帰納バイアス', fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_curriculum_map()

In [ ]:
def summarize_key_concepts():
    """各セクションの核心概念をまとめる"""
    print("="*70)
    print("Unit 0.4 核心概念のまとめ")
    print("="*70)
    
    print("""
【Section A: カーネル物理学】
  核心: 畳み込み = 「周囲を見て自分の値を決める」演算
  
  • 数学的定義: (f * g)[n] = Σ f[k] g[n-k]
  • 古典フィルタ: ぼかし、エッジ検出、シャープニング
  • 実装: sliding_window_view, im2col で効率化

【Section B: 受容野】
  核心: 1ピクセルの決定に影響する入力領域
  
  • 漸化式: RF_n = RF_{n-1} + (k_n - 1) × 累積ストライド
  • 深さとの関係: 層を深くすると受容野が指数的に拡大
  • 3DGSとの類似: スプラットサイズ ≈ 受容野

【Section C: 帰納バイアス】
  核心: CNNは「画像には局所的なパターンがある」と仮定
  
  • 重み共有: 位置に依存しない特徴検出
  • 並進等変性: 入力をずらすと出力もずれる
  • 限界: 回転、長距離依存、テクスチャバイアス

【Section D: セグメンテーション】
  核心: ピクセルごとの分類 = 空間情報の完全活用
  
  • U-Net: エンコーダ-デコーダ + スキップ接続
  • FPN: マルチスケール特徴ピラミッド
  • スキップ接続: 加算型（ResNet）vs 連結型（U-Net）
    """)

summarize_key_concepts()

<a id="section2"></a>
## 2. 核心概念の統合

In [ ]:
def visualize_concept_connections():
    """概念間のつながりを可視化"""
    fig, ax = plt.subplots(figsize=(16, 10))
    
    # 中心概念
    concepts = {
        '空間的帰納バイアス': (0.5, 0.5, 'gold', 0.12),
        '局所性': (0.25, 0.75, 'lightblue', 0.08),
        '重み共有': (0.75, 0.75, 'lightgreen', 0.08),
        '受容野': (0.25, 0.25, 'lightcoral', 0.08),
        'セグメンテーション': (0.75, 0.25, 'lavender', 0.08),
    }
    
    for name, (x, y, color, radius) in concepts.items():
        circle = plt.Circle((x, y), radius, facecolor=color, edgecolor='gray', linewidth=2)
        ax.add_patch(circle)
        ax.text(x, y, name, ha='center', va='center', fontsize=10, fontweight='bold')
    
    # 接続線
    connections = [
        ('局所性', '空間的帰納バイアス', '定義の一部'),
        ('重み共有', '空間的帰納バイアス', '定義の一部'),
        ('空間的帰納バイアス', '受容野', '設計に影響'),
        ('空間的帰納バイアス', 'セグメンテーション', '応用'),
        ('局所性', '受容野', '関連'),
        ('重み共有', 'セグメンテーション', '効率化'),
    ]
    
    for c1, c2, label in connections:
        x1, y1, _, r1 = concepts[c1]
        x2, y2, _, r2 = concepts[c2]
        ax.plot([x1, x2], [y1, y2], 'k-', alpha=0.3, linewidth=2)
        ax.text((x1+x2)/2, (y1+y2)/2, label, fontsize=8, ha='center', va='center',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('概念間のつながり', fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_concept_connections()

In [ ]:
def explain_unified_view():
    """統一的な視点からの説明"""
    print("="*70)
    print("統一的視点：なぜCNNは画像に効くのか")
    print("="*70)
    
    print("""
【根本的な問い】
  なぜCNNは画像認識で成功したのか？

【答え：帰納バイアスが画像の性質と一致】

  ┌─────────────────────────────────────────────────────┐
  │ 画像の性質          →    CNNの設計原理            │
  ├─────────────────────────────────────────────────────┤
  │ 局所的なパターン    →    小さなカーネル            │
  │ パターンは位置不変  →    重み共有                  │
  │ 階層的な構造        →    深い層構造                │
  │ スケール変化        →    ダウンサンプリング        │
  └─────────────────────────────────────────────────────┘

【受容野の役割】
  • 浅い層: 小さな受容野 → 局所的な特徴（エッジ、テクスチャ）
  • 深い層: 大きな受容野 → 大域的な特徴（物体全体）
  
  この階層構造が、画像の「部分→全体」の構造と一致

【セグメンテーションへの発展】
  • 分類: 画像 → 1つのラベル（大域的な受容野が必要）
  • セグメンテーション: 画像 → ピクセルごとのラベル（局所と大域の両方）
  
  → U-Netのスキップ接続が「局所と大域」を橋渡し

【限界と次世代】
  • CNNの仮定が成り立たない場合（長距離依存など）
  • → Vision Transformer: 全結合Attention
  • → MLP-Mixer: チャンネルと空間を分離して処理
    """)

explain_unified_view()

<a id="section3"></a>
## 3. 設計原則のまとめ

In [ ]:
def design_principles():
    """CNNアーキテクチャ設計の原則"""
    print("="*70)
    print("CNNアーキテクチャ設計の原則")
    print("="*70)
    
    print("""
【原則1: 受容野を意識する】
  
  目標: 最終層の受容野が入力の重要な部分をカバー
  
  計算式: RF_n = RF_{n-1} + (k_n - 1) × 累積ストライド
  
  例: 3×3カーネル5層、ストライド2が2回
      RF = 1 + 2×1 + 2×1 + 2×2 + 2×2 + 2×4 = 1 + 2 + 2 + 4 + 4 + 8 = 21

【原則2: ダウンサンプリングの戦略】
  
  • プーリング: 位置の微小なずれに頑健
  • Strided Conv: 学習可能なダウンサンプリング
  
  一般的な戦略:
  - 初期層: 解像度を維持（詳細な特徴を抽出）
  - 中間層: 段階的にダウンサンプリング
  - 深い層: 低解像度で大域的な特徴を処理

【原則3: スキップ接続の活用】
  
  目的に応じて選択:
  • 分類: ResNet型（加算）で勾配の流れを改善
  • セグメンテーション: U-Net型（連結）で詳細を保持
  • 両方を組み合わせることも可能

【原則4: チャンネル数の設計】
  
  一般的なパターン:
  • 解像度が半分になるとチャンネル数を2倍
  • 計算量をほぼ一定に保つ
  
  例: 64 → 128 → 256 → 512（解像度: H → H/2 → H/4 → H/8）

【原則5: 正則化と安定化】
  
  • BatchNorm: 各層の出力を正規化
  • Dropout: 過学習を防止
  • Weight Decay: 重みの大きさを制限
    """)

design_principles()

In [ ]:
def visualize_architecture_patterns():
    """代表的なアーキテクチャパターン"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # VGG型
    ax = axes[0]
    layers = [(0.5, 0.9, 0.3, 0.08), (0.5, 0.75, 0.3, 0.08), (0.5, 0.6, 0.25, 0.08),
              (0.5, 0.45, 0.2, 0.08), (0.5, 0.3, 0.15, 0.08), (0.5, 0.15, 0.1, 0.08)]
    
    for x, y, w, h in layers:
        rect = plt.Rectangle((x - w/2, y - h/2), w, h, facecolor='lightblue', edgecolor='blue')
        ax.add_patch(rect)
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('VGG型\n（シンプルな積み重ね）', fontsize=12, fontweight='bold')
    
    # ResNet型
    ax = axes[1]
    for x, y, w, h in layers:
        rect = plt.Rectangle((x - w/2, y - h/2), w, h, facecolor='lightgreen', edgecolor='green')
        ax.add_patch(rect)
    
    # スキップ接続
    for i in range(0, len(layers)-1, 2):
        ax.annotate('', xy=(0.8, layers[i+1][1]), xytext=(0.8, layers[i][1]),
                   arrowprops=dict(arrowstyle='->', color='red', lw=2,
                                  connectionstyle='arc3,rad=0.3'))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('ResNet型\n（スキップ接続で深く）', fontsize=12, fontweight='bold')
    
    # U-Net型
    ax = axes[2]
    encoder = [(0.2, 0.85, 0.15, 0.08), (0.2, 0.65, 0.12, 0.08), (0.2, 0.45, 0.09, 0.08)]
    decoder = [(0.7, 0.45, 0.09, 0.08), (0.7, 0.65, 0.12, 0.08), (0.7, 0.85, 0.15, 0.08)]
    bottleneck = [(0.45, 0.25, 0.06, 0.08)]
    
    for x, y, w, h in encoder:
        rect = plt.Rectangle((x - w/2, y - h/2), w, h, facecolor='lightblue', edgecolor='blue')
        ax.add_patch(rect)
    
    for x, y, w, h in decoder:
        rect = plt.Rectangle((x - w/2, y - h/2), w, h, facecolor='lightgreen', edgecolor='green')
        ax.add_patch(rect)
    
    for x, y, w, h in bottleneck:
        rect = plt.Rectangle((x - w/2, y - h/2), w, h, facecolor='coral', edgecolor='red')
        ax.add_patch(rect)
    
    # スキップ接続
    for enc, dec in zip(encoder, decoder):
        ax.annotate('', xy=(dec[0] - dec[2]/2, dec[1]), 
                   xytext=(enc[0] + enc[2]/2, enc[1]),
                   arrowprops=dict(arrowstyle='->', color='orange', lw=2))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('U-Net型\n（エンコーダ-デコーダ）', fontsize=12, fontweight='bold')
    
    plt.suptitle('代表的なCNNアーキテクチャパターン', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_architecture_patterns()

<a id="section4"></a>
## 4. 理解度チェックリスト

In [ ]:
def understanding_checklist():
    """理解度チェックリスト"""
    print("="*70)
    print("Unit 0.4 理解度チェックリスト")
    print("="*70)
    
    checklist = [
        ("カーネル物理学", [
            "畳み込みを『周囲を見て自分の値を決める演算』と説明できる",
            "畳み込みと相関の違いを説明できる",
            "Sobel、Gaussianフィルタの役割を説明できる",
            "im2colの仕組みを理解している",
        ]),
        ("受容野", [
            "受容野の定義を説明できる",
            "受容野の漸化式を使って計算できる",
            "プーリングとStrided Convの違いを説明できる",
            "受容野と3DGSスプラットの類似性を理解している",
        ]),
        ("帰納バイアス", [
            "帰納バイアスの意味を説明できる",
            "重み共有のメリットを説明できる",
            "並進等変性と並進不変性の違いを説明できる",
            "CNNの限界（回転、長距離依存）を説明できる",
        ]),
        ("セグメンテーション", [
            "セマンティック/インスタンス/パノプティックの違いを説明できる",
            "U-Netの構造とスキップ接続の役割を説明できる",
            "FPNの目的と構造を説明できる",
            "ResNet型とU-Net型のスキップ接続の違いを説明できる",
        ]),
    ]
    
    for section, items in checklist:
        print(f"\n【{section}】")
        for item in items:
            print(f"  □ {item}")
    
    print("\n" + "="*70)
    print("全ての項目にチェックが入れば、Unit 0.4の学習は完了です！")

understanding_checklist()

In [ ]:
def quiz():
    """簡単なクイズ"""
    print("="*70)
    print("クイズ：理解度を確認しよう")
    print("="*70)
    
    questions = [
        ("Q1: 3×3カーネルを5層重ねた時の受容野は？（ストライド全て1）",
         "A1: RF = 1 + 2×5 = 11ピクセル"),
        ("Q2: CNNの『重み共有』によって得られる最大のメリットは？",
         "A2: パラメータ数の大幅削減（位置に関係なく同じ特徴を検出）"),
        ("Q3: U-Netのスキップ接続が『連結』を使う理由は？",
         "A3: 高解像度の詳細情報を保持するため（加算だと情報が混ざる）"),
        ("Q4: Vision TransformerがCNNより優れている点は？",
         "A4: 長距離依存関係を直接モデル化できる（グローバルな注意機構）"),
        ("Q5: FPNの『ラテラル接続』の役割は？",
         "A5: 異なる解像度の特徴を統合（1×1畳み込みでチャンネル数を調整）"),
    ]
    
    for q, a in questions:
        print(f"\n{q}")
        print(f"  → {a}")

quiz()

<a id="summary"></a>
## 5. 次へのステップ

### Unit 0.4で学んだこと

このユニットでは、CNNの空間的帰納バイアスについて深く学びました。

- **畳み込み**は「局所的なパターンを効率的に検出する」演算
- **受容野**は「どれだけの入力領域を見ているか」を表す
- **帰納バイアス**は「事前知識を構造に埋め込む」設計思想
- **セグメンテーション**は「ピクセル単位の空間理解」を実現

### 次のノートブックへ

最後のノートブック（102）では、未来への展望として：
- CNNからTransformerへの進化
- 3D理解（NeRF、3DGS）への接続
- NeoVerseプロジェクトとの関連

を扱います。

In [ ]:
def final_message():
    """最終メッセージ"""
    print("="*70)
    print("おめでとうございます！")
    print("="*70)
    
    print("""
Unit 0.4「空間の帰納バイアス：CNN & 局所性の科学」の
総合演習が完了しました。

【習得したスキル】
  ✓ 畳み込みの直感的・数学的理解
  ✓ 受容野の計算と設計への応用
  ✓ 帰納バイアスの概念と限界の理解
  ✓ セグメンテーションアーキテクチャの設計思想

【次のステップ】
  → ノートブック102: 未来への展望
  → 3DGS/NeRFへの架け橋
  → NeoVerseプロジェクトの基礎完成

あなたは今、深層学習の空間理解における
確かな基礎を身につけました。
    """)

final_message()